In [1]:
import snap
from graphviz import Digraph
import time

In [2]:
# G1 = snap.TUNGraph.New()
# G1.AddNode(0)
# G1.AddNode(1)
# G1.AddNode(2)
# G1.AddNode(3)
# G1.AddNode(4)
# G1.AddNode(5)
# G1.AddNode(6)
# G1.AddNode(7)
# G1.AddNode(8)
# G1.AddNode(9)
# G1.AddNode(10)
# G1.AddEdge(0,1)
# G1.AddEdge(0,2)
# G1.AddEdge(2,3)
# G1.AddEdge(1,3)
# G1.AddEdge(3,4)
# G1.AddEdge(4,5)
# G1.AddEdge(4,6)
# G1.AddEdge(0,7)
# G1.AddEdge(7,8)
# G1.AddEdge(7,9)
# G1.AddEdge(9,10)
# G1

In [3]:
# G1 = snap.TUNGraph.New()
# G1.AddNode(0)
# G1.AddNode(1)
# G1.AddNode(2)
# G1.AddNode(3)
# G1.AddNode(4)
# G1.AddNode(5)
# G1.AddNode(6)
# G1.AddNode(7)
# G1.AddNode(8)
# G1.AddNode(9)
# G1.AddNode(10)
# G1.AddNode(11)
# G1.AddNode(12)
# G1.AddNode(13)
# G1.AddNode(14)
# G1.AddEdge(0,1)
# G1.AddEdge(0,2)
# G1.AddEdge(2,3)
# G1.AddEdge(1,3)
# G1.AddEdge(3,4)
# G1.AddEdge(4,5)
# G1.AddEdge(4,6)
# G1.AddEdge(0,7)
# G1.AddEdge(7,8)
# G1.AddEdge(7,9)
# G1.AddEdge(9,10)
# G1.AddEdge(0,11)
# G1.AddEdge(11,12)
# G1.AddEdge(11,13)
# G1.AddEdge(13,14)
# G1

In [4]:
# G1 = snap.GenRndGnm(snap.TUNGraph, 22, 60) #random graph
#G1 = snap.GenFull(snap.TUNGraph, 4) #complete graph 最佳解都會是保護1個點
#G1 = snap.GenGrid(snap.TUNGraph, 3, 3, False) #Grid graph N8M
#G1 = snap.GenStar(snap.TUNGraph, 5, False) #star topology 最佳解都會是保護1個點
#G1 = snap.GenTree(snap.TUNGraph, 3, 2) #Tree graph (Number of children and level)

In [5]:
#匯出檔案
# snap.SaveEdgeList(G1, "graph.txt", "Save as tab-separated list of edges")

In [6]:
#匯入圖
G1 = snap.LoadEdgeList(snap.TUNGraph, "graph.txt", 0, 1)
start_time = time.time()

In [7]:
#刪除孤單的點
lonely_node = []
for NI in G1.Nodes():
    if NI.GetOutDeg()==0:
        lonely_node.append(NI.GetId())
        G1.DelNode(NI.GetId())

print("lonely_node = ",lonely_node)

lonely_node =  []


In [8]:
# output 圖片 !!!!Data大容易跑很久!!!!
# labels = {}
# for NI in G1.Nodes():
#         labels[NI.GetId()] = str(NI.GetId())
# G1.DrawGViz(snap.gvlDot, "Brute_Force_optimal_valid.png", " ", labels)

In [9]:
AllnodeID = [] #全部的node
InfectednodeID = [0]
vulnerablenodeID = []
vaccinatednodeID = [] #vaccinated的node
# inf_plus_vul_minus_zero = []
for NI in G1.Nodes():
    AllnodeID.append(NI.GetId())
    if(NI.GetId()!=0):
        vulnerablenodeID.append(NI.GetId())
    print("node: %d" % NI.GetId())

node: 0
node: 5
node: 7
node: 13
node: 1
node: 8
node: 9
node: 28
node: 2
node: 10
node: 12
node: 3
node: 19
node: 20
node: 4
node: 15
node: 17
node: 22
node: 29
node: 6
node: 23
node: 24
node: 26
node: 27
node: 11
node: 14
node: 18
node: 21
node: 25
node: 16


In [10]:
#計算圖的層數
layer = 0
for NI in G1.Nodes():
    if(G1.GetShortPath(NI.GetId(),0)>layer):
        layer = G1.GetShortPath(NI.GetId(),0)
layer

4

In [11]:
vulnerablenodeID

[5,
 7,
 13,
 1,
 8,
 9,
 28,
 2,
 10,
 12,
 3,
 19,
 20,
 4,
 15,
 17,
 22,
 29,
 6,
 23,
 24,
 26,
 27,
 11,
 14,
 18,
 21,
 25,
 16]

In [12]:
import itertools #算排列組合
inp_list = vulnerablenodeID
permutations = list(itertools.permutations(inp_list,r=layer))

In [13]:
print(len(permutations))
print(len(permutations[1]))

570024
4


In [14]:
# permutations

In [15]:
# permutations[0][3]

In [16]:
def copy_graph(graph):
    tmpfile = '.copy.bin'

    # Saving to tmp file
    FOut = snap.TFOut(tmpfile)
    graph.Save(FOut)
    FOut.Flush()

    # Loading to new graph
    FIn = snap.TFIn(tmpfile)
    graphtype = type(graph)
    new_graph = graphtype.New()
    new_graph = new_graph.Load(FIn)

    return new_graph

In [17]:
saved = []
save_node = []
S = 0 #初始感染的點
for i in range(len(permutations)):#全部的排列
    G_temp = copy_graph(G1)
    temp_vaccinated = []
    temp_infected = [0]
    temp_vulnerable = vulnerablenodeID.copy()

    for j in range(len(permutations[i])): #依順序注射疫苗
        temp_saved = []

        b = temp_vulnerable.copy()
        if(permutations[i][j] not in temp_vulnerable): #如果要注射的點已經被保護了(從圖中刪除)
            for n in range(len(b)):
                if(G_temp.GetShortPath(S,b[n])==j+1):#病毒開始擴散
                    temp_infected.append(b[n])
                    temp_vulnerable.remove(b[n])

            continue
        for k in range(len(temp_vulnerable)):#每一個點計算是否有被注射的點保護
#             if(j==0):
#                 r = 1
#             else:
#                 r = j
            r  = j+1

            if(r<=(G_temp.GetShortPath(S,temp_vulnerable[k])-G_temp.GetShortPath(permutations[i][j],temp_vulnerable[k]))):
                temp_saved.append(temp_vulnerable[k])

                
        for m in range(len(temp_saved)): #調整位置
            temp_vaccinated.append(temp_saved[m])
            temp_vulnerable.remove(temp_saved[m])
            G_temp.DelNode(temp_saved[m])
            
        a = temp_vulnerable.copy()
        for n in range(len(a)):#病毒開始擴散
            if(G_temp.GetShortPath(S,a[n])==j+1):
                temp_infected.append(a[n])
                temp_vulnerable.remove(a[n])

    save_node.append(temp_vaccinated)
    saved.append(len(temp_vaccinated))
                   

In [18]:
end_time = time.time()

In [19]:
print("最佳策略=",permutations[saved.index(max(saved))])
print("保護個數=",max(saved))
print("被保護的點=",save_node[saved.index(max(saved))])
print("計算時間=",end_time-start_time,"sec")

最佳策略= (7, 15, 9, 5)
保護個數= 20
被保護的點= [7, 1, 8, 28, 2, 10, 12, 3, 19, 20, 23, 24, 26, 27, 21, 16, 4, 15, 14, 9]
計算時間= 2526.7300589084625 sec
